# CPE494/663 GENERATIVE ARTIFICIAL INTELLIGENCE
## Lab 1: PyTorch Discriminative Model

### Download dataset

Using `RealWaste` dataset from https://archive.ics.uci.edu/dataset/908/realwaste  
The dataset contains 4752 images of waste items across 9 material types.

In [ ]:
import requests
import zipfile
from pathlib import Path
from io import BytesIO

In [ ]:
%%time
if not Path("realwaste-main").is_dir():
    print("Downloading dataset...")
    resp = requests.get("https://archive.ics.uci.edu/static/public/908/realwaste.zip")
    content = BytesIO(resp.content)
    with zipfile.ZipFile(content, 'r') as zip_ref:
        zip_ref.extractall(".")
    print("Downloaded and extracted dataset to `./realwaste-main`")
else:
    print("Dataset already exists")

### Define `datasets` and `DataLoader`

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets

In [ ]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define data transformation
data_transforms = transforms.Compose([transforms.Resize((256, 256)),
                                      transforms.ToTensor()
                                     ])

In [ ]:
# Read image directory as dataset
realwaste_img = datasets.ImageFolder("realwaste-main/RealWaste/", 
                                     transform=data_transforms)

In [ ]:
# Get number of classes in the dataset
num_classes = len(realwaste_img.classes)
print(f"Dataset contains {num_classes} classes: {realwaste_img.classes}")

In [ ]:
# Split dataset into training and test set
train_ratio = 0.7

train_size = int(train_ratio * len(realwaste_img))
test_size = len(realwaste_img) - train_size

train_dataset, test_dataset = random_split(realwaste_img, 
                                           [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
# Show sample image from training set
import matplotlib.pyplot as plt
import numpy as np

def show_image(img_tensor):
    img = img_tensor.numpy().transpose((1, 2, 0))
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

images, labels = next(iter(train_loader))
show_image(images[0])
print(f"Label: {realwaste_img.classes[labels[0]]}")

### Modeling

**Your turn:** Develop and train a discriminative neural network to classify images within the `RealWaste` dataset.  
You can use any neural network architecture; however, you must provide a technical justification for your design choices.  
Evaluate the model's performance using appropriate loss functions and metrics.  

**What to submit:** 2 files: 
1. A Python notebook (.ipynb)
2. Export your notebook as a PDF file
   
Submit to `Lab 1 - PyTorch` activity in LEB2.  


**Due date:** 26/01/2026 17:59

In [ ]:
# Work here

---